In [ ]:
# Descargamos la base de datos
# https://www.kaggle.com/benvozza/spam-classification/data?select=spam.csv

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import string
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "/content"]).decode("utf8"))

# Any results you write to the current directory are saved as output.
data = pd.read_csv("/content/spam.csv",encoding='latin-1')

data.head()

sample_data
spam.csv



,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


# Limpiamos los datos

In [ ]:
data = data.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
data = data.rename(columns={"v1":"class", "v2":"text"})
data.head()

,class,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


# Obtememos la longitud de cada email

In [ ]:
data['length'] = data['text'].apply(len)
data.head()

,class,text,length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [ ]:
def pre_process(text):
    # Retiramos signos de puntuación        
    text = text.translate(str.maketrans('', '', string.punctuation))    
    # Retiramos las stopwords
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    words = ""
    # Aplicamos radicalización (stemming)
    for i in text:
            stemmer = SnowballStemmer("english")
            words += (stemmer.stem(i))+" "            
    return words

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
textFeatures = data['text'].copy()
# Preprocesamos texto
textFeatures = textFeatures.apply(pre_process)
# Aplicamos TF-IDF
vectorizer = TfidfVectorizer()
features = vectorizer.fit_transform(textFeatures)

# Separamos en train y test
features_train, features_test, labels_train, labels_test = train_test_split(features, data['class'], test_size=0.3, random_state=111)

In [ ]:
print(features_train.shape)
print(features_test.shape)

(3900, 8158)
(1672, 8158)


In [ ]:
textFeatures

0       go jurong point, crazy.. avail bugi n great wo...
1                            ok lar... joke wif u oni... 
2       free entri 2 wkli comp win fa cup final tkts 2...
3              u dun say earli hor... u c alreadi say... 
4                  nah think goe usf, live around though 
                              ...                        
5567    2nd time tri 2 contact u. u å£750 pound prize....
5568                           ì_ b go esplanad fr home? 
5569            pity, * mood that. so...ani suggestions? 
5570    guy bitch act like i'd interest buy someth els...
5571                                     rofl. true name 
Name: text, Length: 5572, dtype: object

# Clasificador

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

svc = SVC(kernel='sigmoid', gamma=1.0)
svc.fit(features_train, labels_train)
prediction = svc.predict(features_test)
accuracy_score(labels_test,prediction)

0.9802631578947368

# Tarea 

Utilizar otros clasificadores para evaluar el rendimiento del sistema.